## Workflow:
-Get Inference Results from BERT Model
-Run this notebook on each chunk to combine with dock

In [ ]:
#Set Chunk Here
chunk= 

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 300)

In [2]:
columns_to_load = ['id', 'court_id']  # Adjust this list based on your actual column names

try:
    # Reading the CSV file into a DataFrame
    filtered_docket_df = pd.read_csv('/vast/amr10211/dockets-2023-08-31-filtered-withna.csv', 
                                     usecols=columns_to_load, 
                                     error_bad_lines=False,
                                     warn_bad_lines=True)
except pd.errors.ParserError as e:
    print(f"ParserError: {e}")

/state/partition1/job-40555904/ipykernel_1065238/121397283.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  filtered_docket_df = pd.read_csv('/vast/amr10211/dockets-2023-08-31-filtered-withna.csv',
/state/partition1/job-40555904/ipykernel_1065238/121397283.py:5: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  filtered_docket_df = pd.read_csv('/vast/amr10211/dockets-2023-08-31-filtered-withna.csv',


In [3]:
columns_to_load = ['id', 'docket_id']

cluster_df =  pd.read_csv('/vast/amr10211/opinions-cluster-data-lc.csv', usecols=columns_to_load)

In [44]:
pd.read_csv('/scratch/amh9750/capstone/bert_inference/inference_results_legal_BERT_chunk_1.csv')

/share/apps/python/3.8.6/intel/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (14,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [45]:
inference_df = inference_df.rename(columns={'id': 'opinion_id'})
cluster_df = cluster_df.rename(columns={'id': 'cluster_id'})

In [46]:
inference_df = inference_df.merge(cluster_df[['cluster_id','docket_id']], how='left', on='cluster_id')

In [47]:
inference_df = inference_df.merge(filtered_docket_df, how='left', left_on='docket_id', right_on='id')

In [48]:
inference_df.count()

resource_uri               0
opinion_id            999974
absolute_url               0
cluster_id            999974
cluster                    0
author_id              10833
author                     0
joined_by                  0
date_created          999974
date_modified         999974
author_str            755143
per_curiam            999974
joined_by_str              0
type                  999974
sha1                  168416
page_count             81988
download_url          161539
local_path             83006
extracted_by_ocr      999974
opinions_cited             0
judges                988120
judge                 999974
date_filed            999974
probability           999974
decision_direction    999974
docket_id             999974
id                    257293
court_id              257293
dtype: int64

In [50]:
inference_df.head(5)

,resource_uri,opinion_id,absolute_url,cluster_id,cluster,author_id,author,joined_by,date_created,date_modified,...,extracted_by_ocr,opinions_cited,judges,judge,date_filed,probability,decision_direction,docket_id,id,court_id
0,NaN,7347153,NaN,7427349,NaN,NaN,NaN,NaN,2022-07-26 01:06:24.86499+00,2022-07-26 01:06:24.865016+00,...,t,NaN,Clement,Clement,1954-04-08,0.549185,1,64418478,64418478.0,calappdeptsuper
1,NaN,7842685,NaN,7895231,NaN,NaN,NaN,NaN,2022-09-08 17:04:04.834586+00,2022-09-08 17:04:04.834614+00,...,t,NaN,Berdon,Berdon,1993-01-21,0.566750,1,64988703,NaN,NaN
2,NaN,7845684,NaN,7897737,NaN,NaN,NaN,NaN,2022-09-08 17:10:04.781207+00,2022-09-08 17:10:04.781234+00,...,t,NaN,Berdon,Berdon,1998-07-16,0.591329,1,64991247,NaN,NaN
3,NaN,7347154,NaN,7427350,NaN,NaN,NaN,NaN,2022-07-26 01:06:24.916646+00,2022-07-26 01:06:24.916666+00,...,t,NaN,Hilliard,Hilliard,1954-04-12,0.440153,0,64418479,64418479.0,calappdeptsuper
4,NaN,7347207,NaN,7427399,NaN,NaN,NaN,NaN,2022-07-26 01:06:32.636426+00,2022-07-26 01:06:32.636445+00,...,t,NaN,Bishop,Bishop,1957-10-07,0.511651,1,64418528,64418528.0,calappdeptsuper


In [51]:

# List of columns to check for null values
columns_to_check = ['author_str', 'author_id', 'court_id']

# Count the number of null values for each row across the specified columns
null_counts_per_row = inference_df[columns_to_check].isnull().sum(axis=1)

# Count the number of rows where all specified columns are null
total_rows_with_all_null = (null_counts_per_row == len(columns_to_check)).sum()

# Display the result
print("Total rows where all three columns are null:", total_rows_with_all_null)


Total rows where all three columns are null: 175951


In [52]:
inference_df.to_csv('lower_court_data_labels_chunk1_with_courtID.csv')